In [ ]:
%matplotlib widget
# make sure prints in callbacks make it to the notebook

# See https://ipywidgets.readthedocs.io/en/stable/examples/Output%20Widget.html

# By default, calling `print` in a ipywidgets callback results in the output
# being lost (because it is not clear _where_ it should go).  You can explictily
# capture that the text to a given output area using at Output widget.

# This is a wrapper for `plt.subplots` that makes sure
#   a) an ipywidgets.widgets.Output is created with each Figure
#   b) the `mpl_connect` on the canvas is monkey-patched such that all
#      user callbacks run in a context where the stdout is captured and sent
#      to that output area.
import matplotlib.pyplot as plt
import functools
@functools.wraps(plt.figure)
def figure(*args, **kwargs):
    from IPython.display import display
    import ipywidgets as widgets
    import weakref
    import functools
    
    fig  = figure._figure(*args, **kwargs)
    fig._output = output = widgets.Output()
    display(output)
    
    orig_mpl_connect = fig.canvas.mpl_connect
    
    @functools.wraps(orig_mpl_connect)
    def mpl_connect(key, cb, **kwargs):
        # try to use a WeakMethod to make sure we don't keep objects alive
        # to match the behavior of the base mpl_connect
        try:
            r = weakref.WeakMethod(cb)
        except TypeError:
            r = lambda: cb
        def wrapper(*args, **kw):
            cb = r()
              
            with output:
                if cb is not None:
                    cb(*args, **kw)
                
        orig_mpl_connect(key, wrapper, **kwargs)
    
    # mokeny patch the canvas 
    fig.canvas.mpl_connect = mpl_connect
    return fig
figure._figure = plt.figure
# monkey patch pyplot (!?)
plt.figure = figure

In [ ]:
import seaborn as sns
mpg = sns.load_dataset('mpg')

In [ ]:
import matplotlib.widgets as mwidgets
import numpy as np
def make_slider(fig, data): 
    x_data = 'model_year' 
    y_data = 'acceleration' 
    c_data = 'horsepower' 
    s_data = 'displacement' 
     
    h_data = 'mpg' 
    ax1, ax2 = fig.subplots(2, 1) 
    hist = ax2.hist(h_data, data=data, bins='auto') 
    ax2.set_xlabel(h_data) 
    ax2.set_ylabel('N') 
    range_label = ax2.annotate('', (1, 1), 
                               ha='right', va='top', 
                               xycoords='axes fraction', 
                               xytext=(-4, -4), 
                               textcoords='offset points') 
 
    sc = ax1.scatter(x_data, y_data, c=c_data, s=s_data, data=data, 
                     alpha=.5, linewidth=3) 
    ax1.set_xlabel(x_data) 
    ax1.set_ylabel(y_data) 
    # produce a legend with the unique colors from the scatter 
    legend1 = ax1.legend(*sc.legend_elements(), 
                    loc="upper left", title=c_data,  
                   ncol=3) 
    ax1.add_artist(legend1) 
     # produce a legend with a cross section of sizes from the scatter 
    handles, labels = sc.legend_elements(prop="sizes", alpha=0.6) 
    legend2 = ax1.legend(handles, labels, loc="lower right", title=s_data, ncol=3) 
         
    base_ec = None 
    def mark_markers(low, high): 
        nonlocal base_ec 
        if base_ec is None: 
            base_ec = sc.get_edgecolors() 
        ec = np.array(base_ec) 
        mask = (low < data[h_data]) & (data[h_data] < high) 
        mask = mask.values 
        ec[mask, :] = [1, 0, 0, 1] 
        sc.set_edgecolor(ec) 
        range_label.set_text(f'showing {low:.2f} < {h_data} < {high:.2f}')
        fig.canvas.draw_idle()
         
    slider = mwidgets.SpanSelector(ax2, mark_markers, 'horizontal', useblit=True, span_stays=True) 
    return slider 
 
s = make_slider(figure(figsize=(16, 8)), mpg)